In [1]:
import numpy as np
from scipy.io import loadmat
import matplotlib.pyplot as plt
import pandas as pd
from scipy.linalg import null_space
%matplotlib qt

In [2]:
def ComputeReprojectionError(P_1, P_2, X_j, x_1j, x_2j):
    r_1 = np.array([
        x_1j[0] - (P_1[0, :] @ X_j) / (P_1[2, :] @ X_j),
        x_1j[1] - (P_1[1, :] @ X_j) / (P_1[2, :] @ X_j)
    ])
    r_2 = np.array([
        x_2j[0] - (P_2[0, :] @ X_j) / (P_2[2, :] @ X_j),
        x_2j[1] - (P_2[1, :] @ X_j) / (P_2[2, :] @ X_j)
    ])
    rs = np.hstack((r_1, r_2))
    error = np.linalg.norm(r_1)**2 + np.linalg.norm(r_2)**2
    return error, rs

def LinearizeReprojErr(P_1, P_2, X_j, x_1j, x_2j):
    _, rs = ComputeReprojectionError(P_1, P_2, X_j, x_1j, x_2j)

    J_1 = np.zeros((2, 4))
    J_2 = np.zeros((2, 4))

    J_1[0, :] = (((P_1[0, :] @ X_j) / (P_1[2, :] @ X_j)**2) * P_1[2, :]) - ((1 / (P_1[2, :] @ X_j)) * P_1[0, :])
    J_1[1, :] = (((P_1[1, :] @ X_j) / (P_1[2, :] @ X_j)**2) * P_1[2, :]) - ((1 / (P_1[2, :] @ X_j)) * P_1[1, :])

    J_2[0, :] = (((P_2[0, :] @ X_j) / (P_2[2, :] @ X_j)**2) * P_2[2, :]) - ((1 / (P_2[2, :] @ X_j)) * P_2[0, :])
    J_2[1, :] = (((P_2[1, :] @ X_j) / (P_2[2, :] @ X_j)**2) * P_2[2, :]) - ((1 / (P_2[2, :] @ X_j)) * P_2[1, :])

    J = np.vstack((J_1, J_2))
    return rs, J

def ComputeUpdate(r, J, mu):
    JTJ = J.T @ J
    JTr = J.T @ r
    delta_X_j = -(np.linalg.inv(JTJ + mu * np.eye(JTJ.shape[0])) @ JTr)
    return delta_X_j

def Refine3DPoints(P_1, P_2, X, x_1, x_2, max_iterations=50, initial_mu=0.1):
    n = X.shape[0]
    m = X.shape[1]
    refined_X = np.zeros((n, m))
    mu = initial_mu

    for j in range(m):
        X_j = X[:, j]
        x_1j = x_1[:, j]
        x_2j = x_2[:, j]

        prev_error, _ = ComputeReprojectionError(P_1, P_2, X_j, x_1j, x_2j)

        for i in range(max_iterations):
            residuals, J = LinearizeReprojErr(P_1, P_2, X_j, x_1j, x_2j)
            delta_X_j = ComputeUpdate(residuals, J, mu)
            updated_X_j = X_j + delta_X_j
            updated_error, _ = ComputeReprojectionError(P_1, P_2, updated_X_j, x_1j, x_2j)

            if updated_error < prev_error:
                X_j = updated_X_j
                prev_error = updated_error
                mu /= 10
            else:
                mu *= 10

            mu = max(min(mu, 1e10), 1e-10)

        refined_X[:, j] = X_j

    return refined_X

def add_noise(data, sigma):
    noise = np.random.normal(0, sigma, data.shape)
    return data + noise

def empirical_noise_analysis(P_1, P_2, X, x_1, x_2, noise_levels_3D, noise_levels_2D):

    results = []

    for sigma_X in noise_levels_3D:
        for sigma_x in noise_levels_2D:
            print(f"\nTesting with σX={sigma_X}m and σx={sigma_x}px")

            noisy_X = add_noise(X, sigma_X)
            noisy_x_1 = add_noise(x_1, sigma_x)
            noisy_x_2 = add_noise(x_2, sigma_x)

            error_before = sum([ComputeReprojectionError(P_1, P_2, noisy_X[:, j], noisy_x_1[:, j], noisy_x_2[:, j])[0] for j in range(X.shape[1])])
            median_error_before = np.median([ComputeReprojectionError(P_1, P_2, noisy_X[:, j], noisy_x_1[:, j], noisy_x_2[:, j])[0] for j in range(X.shape[1])])

            refined_X = Refine3DPoints(P_1, P_2, noisy_X, noisy_x_1, noisy_x_2)

            error_after = sum([ComputeReprojectionError(P_1, P_2, refined_X[:, j], noisy_x_1[:, j], noisy_x_2[:, j])[0] for j in range(X.shape[1])])
            median_error_after = np.median([ComputeReprojectionError(P_1, P_2, refined_X[:, j], noisy_x_1[:, j], noisy_x_2[:, j])[0] for j in range(X.shape[1])])

            results.append({
                "σX (m)": sigma_X,
                "σx (px)": sigma_x,
                "Error Before LM": error_before,
                "Median Error Before LM": median_error_before,
                "Error After LM": error_after,
                "Median Error After LM": median_error_after
            })

            print(f"Error Before LM: {error_before}, Median Error Before LM: {median_error_before}")
            print(f"Error After LM: {error_after}, Median Error After LM: {median_error_after}")

            fig = plt.figure()
            ax = fig.add_subplot(111, projection='3d')
            ax.scatter(noisy_X[0, :], noisy_X[1, :], noisy_X[2, :], s=5, c="b", label="Noisy Original Points", edgecolors="b", facecolors="b")
            ax.scatter(refined_X[0, :], refined_X[1, :], refined_X[2, :], s=20, edgecolors="r", facecolors="none", label="Refined Points", linewidth=0.8)
            ax.set_title(f"3D Points for Sigma_X={sigma_X}, Sigma_x={sigma_x}")
            ax.set_xlabel("X")
            ax.set_ylabel("Y")
            ax.set_zlabel("Z")
            ax.legend()
            plt.show()

    return results

In [3]:
data = loadmat('data/compEx3data')
P = data["P"][0]
P_1 = P[0]
P_2 = P[1]
X = data["X"]
x = data["x"][0]
x_1 = x[0]
x_2 = x[1]

noise_levels_3D = [0, 0.1]  
noise_levels_2D = [0, 3] 

results = empirical_noise_analysis(P_1, P_2, X, x_1, x_2, noise_levels_3D, noise_levels_2D)

df_results = pd.DataFrame(results)
print("\nNoise Sensitivity Analysis Results:")
print(df_results)



Testing with σX=0m and σx=0px
Error Before LM: 22354.24499862846, Median Error Before LM: 11.659921338311044
Error After LM: 21566.419995521854, Median Error After LM: 11.19789965906423

Testing with σX=0m and σx=3px
Error Before LM: 89640.64261073712, Median Error Before LM: 39.1472784696666
Error After LM: 38309.5326302039, Median Error After LM: 11.810780805679782

Testing with σX=0.1m and σx=0px
Error Before LM: 891619492.8615746, Median Error Before LM: 315597.38236296363
Error After LM: 21566.41999552174, Median Error After LM: 11.197899659064667

Testing with σX=0.1m and σx=3px
Error Before LM: 874501592.9481186, Median Error Before LM: 324842.50415575807
Error After LM: 38980.406969331816, Median Error After LM: 10.97735361250863

Noise Sensitivity Analysis Results:
   σX (m)  σx (px)  Error Before LM  Median Error Before LM  Error After LM  \
0     0.0        0     2.235424e+04               11.659921    21566.419996   
1     0.0        3     8.964064e+04               39.147